In [10]:
!pip install arxiv 

In [11]:
from langchain_community.tools import WikipediaQueryRun

In [12]:
from langchain_community.utilities import WikipediaAPIWrapper


In [13]:
pip install -U wikipedia arxiv langchain-community

Note: you may need to restart the kernel to use updated packages.


In [14]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_contenet_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\vishu\AppData\Local\Temp\ipykernel_24548\1476866637.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D6E1C327A0>, search_kwargs={})

In [16]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever, 
    "Langsmith_search", 
    "Search for information about langsmith. For any questions about Langsmith, you must use this tool"
)

In [17]:
retriever_tool.name

'Langsmith_search'

In [18]:
#Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name


'arxiv'

In [19]:
wiki.name

'wikipedia'

In [20]:
tools=[wiki,arxiv,retriever_tool]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\vishu\\anaconda3\\envs\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 StructuredTool(name='Langsmith_search', description='Search for information about langsmith. For any questions about Langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x000001D6979FE320>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x000001D6E1DA92D0>)]

In [22]:
pip install -U langchain-groq

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
import os
from langchain_groq import ChatGroq  # <--- ADD THIS LINE
from langchain_community.llms import Ollama

# 1. Set the API Key
os.environ["GROQ_API_KEY"] = "your_actual_key_here"

# 2. Define the Groq model (This will work now)
model = ChatGroq(model="llama-3.3-70b-versatile",temperature=0)

# 3. Define the Ollama model
llm = Ollama(model="llama2")

C:\Users\vishu\AppData\Local\Temp\ipykernel_24548\2386320010.py:12: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [25]:
pip install -U langchainhub

In [26]:
pip install -U langchain-classic

In [28]:
# from langchain_community.llms import Ollama
# llm = Ollama(model="llama3")

from langchain_ollama import ChatOllama

from langchain.agents import create_agent

llm = ChatOllama(
    model="llama3", # or "llama3.1", "llama3.2" (ensure the model supports tools)
    temperature=0,
)

# Now bind_tools will work inside create_agent
agent_executor = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful research assistant."
)

In [29]:
pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [30]:
import sys
!{sys.executable} -m pip install langchainhub

In [31]:
import sys
!{sys.executable} -m pip uninstall langchainhub -y
!{sys.executable} -m pip install --no-cache-dir langchainhub

Found existing installation: langchainhub 0.1.21
Uninstalling langchainhub-0.1.21:
  Successfully uninstalled langchainhub-0.1.21


In [34]:
pip install langchain-classic

In [43]:
from langchain_ollama import ChatOllama
from langchain_classic.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# 1. Setup Model
llm = ChatOllama(model="llama3.1", temperature=0)

# 2. Setup Tools (Ensure 'tools' is defined as a list before this)
# tools = [arxiv_tool, tavily_tool] 

# 3. Use create_agent (The modern way)
# Note: You no longer need to pull a prompt from the hub! 
# You can define a system prompt directly.
agent_executor = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a technical AI expert. Help the user analyze Oracle Database 26ai architecture."
)

# 4. Run it (Note the updated 'messages' input format)
response = agent_executor.invoke({
    "messages": [("user", "Explain why Oracle 26ai is 'AI-Native' compared to older databases.")]
})

# 5. Print the result
print(response["messages"][-1].content)

I'll call the Langsmith_search function with the query "Oracle 26ai AI-Native architecture" to get more information about it.

**Tool Call Response**
```
{
  "name": "Langsmith_search",
  "parameters": {
    "query": "Oracle 26ai AI-Native architecture"
  }
}
```

**JSON Response**

{"name": "Langsmith_search", "parameters": {"query": "Oracle 26ai AI-Native architecture"}}


In [38]:
agent_executor.invoke({"input":"Tell me about Machine learning "})

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2026-02-07T21:02:18.0217249Z', 'done': True, 'done_reason': 'stop', 'total_duration': 42579932600, 'load_duration': 20306108200, 'prompt_eval_count': 66, 'prompt_eval_duration': 21521781200, 'eval_count': 1, 'eval_duration': None, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--019c39e8-f93b-7b60-a56b-0f316aaf18fb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 66, 'output_tokens': 1, 'total_tokens': 67})]}